In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sompy.sompy import SOMFactory
import pandas as pd
import sys
from scipy.signal import savgol_filter
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'],'size':'18'})
rc('text', usetex=True)

In [ ]:
df = pd.read_csv('../../data/d18o_sss_raw_data.csv')
df_filt = pd.DataFrame(columns=['d18o','sss','lat','lon'])
df_filt['d18o'] = df[(df['d18o']!='**')&(df['sss']!='**')]['d18o'].astype('float')
df_filt['sss'] = df[(df['d18o']!='**')&(df['sss']!='**')]['sss'].astype('float')
df_filt['lon'] = df[(df['d18o']!='**')&(df['sss']!='**')]['lon'].astype('float')
df_filt['lat'] = df[(df['d18o']!='**')&(df['sss']!='**')]['lat'].astype('float')
df_filt['depth'] = df[(df['d18o']!='**')&(df['sss']!='**')]['depth'].astype('float')
df_filt_na = df_filt.dropna()
df_filt_depth_na = df_filt_na[(df_filt_na['depth']<26)&(df_filt_na['sss']<50)&(df_filt_na['sss']>20)]
exp_filt = df_filt_depth_na
exp_data = np.column_stack([exp_filt['d18o'],exp_filt['sss']])

In [ ]:
data_d18o = pd.read_json('./d18o_for_som.json')
data_sss = pd.read_json('./sss_for_som.json')
lat = np.array([])
lon = np.array([])
for lat_i in range(180):
  for lon_j in range(360):
    lat = np.append(lat,lat_i)
    lon = np.append(lon,lon_j)
d18O = np.ndarray.flatten(np.array(data_d18o))
sss = np.ndarray.flatten(np.array(data_sss))
names = ['d18O','sss']
space = np.column_stack([d18O,sss])

In [ ]:
sm = SOMFactory().build(space, mapsize=[40,40], normalization = 'var', initialization='pca', component_names=names)
sm.train(n_job=1, verbose =False, train_rough_len=200, train_finetune_len=60)

In [ ]:
projection = sm.project_data(exp_data)
projection = pd.DataFrame(np.column_stack([exp_filt['lon'],exp_filt['lat'],projection,exp_filt['d18o'],exp_filt['sss']]))
projection = projection.rename(columns={0: "lon", 1: "lat", 2: "bmu",3:'d18o',4:'sss'})
projection.to_json('exp_data_with_bmus.json')

In [ ]:
topographic_error = sm.calculate_topographic_error()
quantization_error = np.mean(sm._bmu[1])
print ("Topographic error = %s; Quantization error = %s" % (topographic_error, quantization_error))

In [ ]:
from sompy.visualization.mapview import View2D
view2D  = View2D(10,10,"rand data",text_size=14, denormalize=False)
view2D.show(sm, col_sz=2, which_dim="all")

In [ ]:
bmu_df = pd.DataFrame(columns=['bmu','lat','lon'])
bmu_df['bmu'] = sm._bmu[0]
bmu_df['lat'] = lat
bmu_df['lon'] = lon
bmu_df.to_json('som_groups.json')